In [ ]:
# java version 8로 변경
!java -version
!sudo update-alternativeAs --config java
!java -version
# 그래프에서 한글표현을 위해 폰트 설치
%config InlineBackend.figure_format = 'retina'
!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

# 기본 글꼴 변경
import matplotlib as mpl
mpl.font_manager._rebuild()
mpl.pyplot.rc('font', family='NanumBarunGothic')
# tensorflow version을 2.x로 변경
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
# 자동 reload
%load_ext autoreload
%autoreload 2
# 구글 드라이브와 연동
from google.colab import drive
drive.mount('/content/drive')
!pip3 install pickle5

openjdk version "11.0.14.1" 2022-02-08
OpenJDK Runtime Environment (build 11.0.14.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.14.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
sudo: update-alternativeAs: command not found
openjdk version "11.0.14.1" 2022-02-08
OpenJDK Runtime Environment (build 11.0.14.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.14.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


Mounted at /content/drive
     |████████████████████████████████| 256 kB 4.9 MB/s 


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS, ALSModel
from tensorflow.keras.applications import MobileNetV2
from tqdm import tqdm
from scipy.spatial.distance import cosine
import pickle5
import joblib

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 50.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ab32c7e5f9b033e95efaff06ae990fda20d5d18cce57eceb07be6774cc367ba4
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
hashtag_metadata=joblib.load('/content/drive/MyDrive/insta_crawling/preprocessed/dataframe/insta_df.pkl')

In [ ]:
# index 재정렬
pd.DataFrame.reset_index(hashtag_metadata, drop=True, inplace=True)

In [ ]:
# hashtag_metadata의 hashtags열에서 '#'으로 시작하는 해시태그만 저장
hashtag_metadata['해시태그'] = hashtag_metadata['해시태그'].apply(
    lambda hashtag_list: [h for h in hashtag_list if h.startswith('#')])

# 2중 리스트를 flatten하게 만들어 모든 해시태그들을 담은 1차원 리스트 생성
all_hashtags = [hashtag for hashtags in hashtag_metadata['해시태그'] for hashtag in hashtags]

# 중복 항목을 제거하기 위해 집합으로 변경하고 다시 리스트로 변경 후 정렬
all_hashtags = sorted(list(set(all_hashtags)))

# hashtag 이름마다 번호를 매기기 위해 lookup 생성
hashtag_lookup = {hashtag: i for i, hashtag in enumerate(all_hashtags)}

In [ ]:
hashtag_rec_data = []
for i in hashtag_metadata.index:
    hashtag_list = hashtag_metadata.loc[i, '해시태그']
    for hashtag in hashtag_list:
        hashtag_rec_data.append(
            {'image_id': i, # hashtag_metadata의 해당 index 번호를 image_id로 부여
             'hashtag_id': hashtag_lookup[hashtag], # hashtag_lookup에 있는 해당 해시태그 번호를 hashtag_id로 부여
             'rating': 1}
        )
hashtag_rec_data = pd.DataFrame(hashtag_rec_data)

## **dnn모델 만들기!!!!!**

In [ ]:
img_shape = (160, 160, 3)

# 사전 학습된 MobileNetV2 모델을 base_model로 저장
base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')
# ***dropout등 튜닝을 해도 될거같다!
# include_top: 네트워크의 최상단에 완전연결 레이어를 넣을지 여부
# 참고로 마지막 분류 계층 ("상단")은 피쳐 추출에 그리 유용하지 않으므로 include_top=False를 지정해 맨 위에 분류 계층을 제외
# weights: 'imagenet' (ImageNet에 대한 선행 학습)

# tf.keras.layers.GlobalAveragePooling2D 레이어를 사용하여 피처를 이미지 당 하나의 1280 요소 벡터로 변환
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# base_model에 global_average_layer 쌓기
neural_network = tf.keras.Sequential([
  base_model,
  global_average_layer,
])
# joblib.dump(neural_network,'./model/mobilenetv2.pkl')

9420800/9406464 [==============================] - 0s 0us/step


## **pic df 만들기!!!!!**

In [ ]:
def prepare_image(img_path, height=160, width=160):
  # 신경망에 맞게 이미지를 다운 샘플링 및 스케일링
  img = tf.io.read_file(img_path) # 불러(읽어)오기
  img = tf.image.decode_jpeg(img) # [height, width, num_channels]인 3차원 배열을 반환
  img = tf.cast(img, tf.float32) # 정수형으로 바꾼경우 소수점을 버린다 boolean일때는 True면 1, False면 0을 출력
  img = (img/127.5) - 1
  img = tf.image.resize(img, (height, width))
  # 컬러 이미지의 차원에 맞게 회색조 이미지 형태변경
  if img.shape != (160, 160, 3):
    img = tf.concat([img, img, img], axis=2)
  return img

def extract_features(image, neural_network):
  # input받은 이미지를 1280개의 deep feature들로 구성된 벡터로 반환
  image_np = image.numpy() # numpy형태로 변환
  images_np = np.expand_dims(image_np, axis=0) # 차원추가([]를 씌워준다)
  deep_features = neural_network.predict(images_np)[0]
  return deep_features

In [ ]:
# 이미지당 딥피쳐(deep_features)를 추출하고
# 이미지(pic), 검색키워드(hashtag),이미지 파일 이름(name), 피쳐(deep_features)를 보여주는 데이터프레임 생성
pics = []

for i, row in tqdm(hashtag_metadata.iterrows()):
    name = row['이미지파일명']
    hashtag = row['검색키워드']
    img_path = f'/content/drive/MyDrive/insta_crawling/image/{hashtag}/{name}'
    try:
        img = prepare_image(img_path)
        deep_features = extract_features(img, neural_network)
        pics.append({'pic': img, 
                     'hashtag': hashtag, # main_hashtag
                     'name': name, # image_local_name
                     'deep_features': deep_features})
    except Exception as e:
        error_type = type(e).__name__
        if error_type == "NotFoundError":
            pass
        else:
            print(e)

pics = pd.DataFrame(pics)
pics.index = pics['name']
# joblib.dump(pics,'/content/drive/MyDrive/insta_crawling/Hashtag_Team16/modeling (1)/pics_0424_1.pkl')

## **als모델 만들기!!!!!**

In [ ]:
hashtag_rec_data
#추천 데이터 양 확인

In [ ]:
# 기존 SparkSession을 가져 오거나 기존 SparkSession이없는 경우 local에 새 SparkSession을 작성
spark = SparkSession.builder.master('local').appName('all').getOrCreate()
from pyspark.sql import SparkSession
als = ALS(userCol='image_id',
          itemCol='hashtag_id',
          implicitPrefs=True, # implicit dataset의 경우 True
          alpha=40)
als.setSeed(0)

hashtag_spark_df = spark.createDataFrame(hashtag_rec_data)
als_model = als.fit(hashtag_spark_df)
# als_model.write().overwrite().save('/content/drive/MyDrive/insta_crawling/Hashtag_Team16/modeling/als')

## **rec df,hashtags df 만들기!!!!!**

In [ ]:
# 각 이미지마다 10개의 해시태그 생성(=추천)
recs = als_model.recommendForAllUsers(numItems=50).toPandas()

hashtag_index = list(all_hashtags)

# hashtag_id로 hashtag 찾는 함수
def lookup_hashtag(hashtag_id):
    return hashtag_index[hashtag_id]

# hashtag_id_scores에서 (hashtag_id, score)마다 hashtag_id의 hashtag 찾는 함수
def lookup_hashtag_recs(hashtag_id_scores):
    return [lookup_hashtag(hashtag_id) for (hashtag_id, score) in hashtag_id_scores]
    # recommendations열에서 hashtag_id로 hashtag를 찾아 recommended_hashtags열에 저장
recs['recommended_hashtags'] = recs['recommendations'].apply(lookup_hashtag_recs)

recs.index = recs['image_id']

# recs 데이터프레임을 기준으로 hashtag_metadata 데이터프레임과 병합
recs = recs.join(hashtag_metadata, how='left')[['recommendations',
                                                 'recommended_hashtags',
                                                 '해시태그',
                                                 '이미지파일명',
                                                 '검색키워드']]
                            
# recommendations열 삭제
recs.drop('recommendations', axis=1, inplace=True)

# image_factors열 추가
image_factors = als_model.userFactors.toPandas()
image_factors.index = image_factors['id']
recs.join(image_factors);

# recs 데이터프레임과 pics 데이터프레임을 image_local_name열 기준으로 병합
# 즉, deep_features열 추가
recs_deep = recs.join(pics, on='이미지파일명', how='inner')

# dict에서 dataframe으로 변환
hashtags_df = pd.DataFrame.from_dict(hashtag_lookup, orient='index')

hashtags_df = hashtags_df.reset_index()
hashtags_df.columns = ['hashtag', 'id']
hashtags_df.index = hashtags_df['id']
hashtags_df.drop('id', axis=1, inplace=True)

img_features = als_model.userFactors.toPandas()

hashtag_features = als_model.itemFactors.toPandas()

# recs_deep 데이터프레임에서 특정 열만 사용
recs_deep_clean = recs_deep[['이미지파일명', '해시태그', 'deep_features']]

img_features.index = img_features['id']
img_features.drop(['id'], axis=1, inplace=True)

# recs_deep_clean 데이터프레임에 img_features 추가
recommender_df = recs_deep_clean.join(img_features, how='inner')

# joblib.dump(recommender_df,'/content/drive/MyDrive/insta_crawling/Hashtag_Team16/modeling (1)/recommender_df_0424_1.pkl')
# joblib.dump(hashtags_df,'/content/drive/MyDrive/insta_crawling/Hashtag_Team16/modeling (1)/hashtags_df_0424_1.pkl')